# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import logging
import os
import csv
import joblib

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
from azureml.core.webservice import LocalWebservice

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

import urllib.request
from azureml.core.model import Model


# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-automl-experiment'

experiment=Experiment(workspace=ws, name=experiment_name)

print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# Create or Attach an AmlCompute cluster

In [ ]:

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "automl-com-clst" #automl-com-clst #automl-com-clst

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

# Data

In [ ]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
#key = "BankMarketing Dataset"
key="Maternal_Health_Risk_Data_Set" #"bankmarketing_train" #bankmarketing_train
#bankmarketing_train
description_text = "UCI machine Learning Maternal Health Risk Data Set"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
#https://archive.ics.uci.edu/ml/machine-learning-databases/00639/Maternal%20Health%20Risk%20Data%20Set.csv
#https://archive.ics.uci.edu/ml/machine-learning-databases/00638/Acoustic%20Features.csv

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00639/Maternal%20Health%20Risk%20Data%20Set.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

# Review the Dataset Result

In [ ]:
dataset.take(5).to_pandas_dataframe()

In [ ]:
#TRAIN_d, TEST_d = train_test_split(df, shuffle=True)
TRAIN_d, TEST_d = train_test_split(concat_x_y, test_size=0.2, random_state=24, shuffle=True)
TRAIN_d.to_csv('train.csv',index = False)

ds_new = ws.get_default_datastore()
ds_new.upload_files(files = ['./train.csv'])
#ds_new.upload(src_dir='.',target_path='.')


train_data = Dataset.Tabular.from_delimited_files(path=[(ds_new,('train.csv'))])


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

# Train
#### This creates a general AutoML settings object. Udacity notes: These inputs must match what was used when training in the portal. label_column_name has to be "RiskLevel" for example.

In [ ]:
# TODO: Put your automl settings here
 # "primary_metric" : 'AUC_weighted'
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "n_cross_validations":3,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=train_data,
                             label_column_name="RiskLevel",   
                             #path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             enable_onnx_compatible_models=True
                             **automl_settings
                            )

In [ ]:
# TODO: Submit your experiment
#remote_run = experiment.submit(automl_config)
remote_run = experiment.submit(config= automl_config, show_output= True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()

In [ ]:
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
#import pickle
#with open(best_model_output._path_on_datastore, "rb" ) as f:
#    best_model = pickle.load(f)
#best_model
automl_best_run, automl_fitted_model = remote_run.get_output()
print("===== AutoMl Best Run ========")
print(automl_best_run)
print("\n  ===== AutoMl Best Fitted Model ========")
print(automl_fitted_model)
automl_best_run.get_tags()
automl_best_run_metrics = automl_best_run.get_metrics()

for metric_name in automl_best_run_metrics:
    metric = automl_best_run_metrics[metric_name]
    print("\n",metric_name, metric)
    

#automl_best_run, automl_fitted_model = remote_run.get_output()
#print(hasattr(automl_fitted_model, 'steps'))
#print("\n")
#print(automl_best_run.get_metrics())
#print("\n")
#print(automl_best_run.get_tags())   

In [ ]:
print(automl_fitted_model.steps)

In [ ]:
#TODO: Save the best model using Onnx
#from azureml.automl.runtime.onnx_convert import OnnxConverter

#automl_best_run, automl_fitted_model = remote_run.get_output(return_onnx_model=True)

#OnnxConverter.save_onnx_model(automl_fitted_model, './outputs/Automl_Onnex_Model.onnx')

In [ ]:

#model = best_run.register_model(model_name='best_autoML_run', model_path='outputs/Automl_Model.joblib')

https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=python
####  Workflow for deploying a model 
The workflow is similar no matter where you deploy your model:

<li>Register the model.
<li>Prepare an entry script.
<li>Prepare an inference configuration.
<li>Deploy the model locally to ensure everything works.
<li>Choose a compute target.
<li>Deploy the model to the cloud.
<li>Test the resulting web service.

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

##### Register Model

In [ ]:
from azureml.core import Model

# Register model
automl_best_run.register_model(model_path='outputs/Maternal_Health_Risk_model.pkl', 
                               model_name='automl_Maternal_Health_Risk_model_pickled',
                        tags={'Training context':'Maternal Health Risk Auto ML Model Trained'},
                        properties={'AUC': automl_best_run_metrics['AUC_weighted'], 
                                    'Accuracy': automl_best_run_metrics['accuracy']})

####  List the registered model

In [ ]:
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

#### Prepare an inference configuration.

In [ ]:
from azureml.automl.core.shared import constants
automl_best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'project_environment.yml')

#### Prepare an inference configuration using an entry script.

In [ ]:
from azureml.core.model import InferenceConfig
#from azureml.core.environment import Environment
from azureml.core import Environment

env = Environment(name="project_environment")
inference_config = InferenceConfig(entry_script="score.py",environment=env)

#inference_config = InferenceConfig(
#    environment=env,
#    source_directory="./source_dir",
#    entry_script="./echo_score.py",
#)

#### Deploy the model locally to ensure everything works. Will skip this part

In [4]:
#from azureml.core.webservice import LocalWebservice
#deployment_config = LocalWebservice.deploy_configuration(port=6789)
##Deploy your machine learning model
#service = Model.deploy(
#    ws,
#    "myservice",
#    [model],
#    dummy_inference_config,
#    deployment_config,
#    overwrite=True,
#)
#service.wait_for_deployment(show_output=True)
#print(service.get_logs())

In [ ]:
from azureml.core.webservice import Webservice, AciWebservice
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb = 1)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:

service=Model.deploy(workspace=ws,
                    name="Maternal-Health-Risk-deploy-service",
                    models=[model],
                    inference_config=inference_config,
                    deployment_config=deployment_config)

service.wait_for_deployment(show_output=True)



TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

service.delete()
model.delete()

In [ ]:
#Define a dummy entry script

#The entry script receives data submitted to a deployed web service and passes it to the model.
#It then returns the model's response to the client. The script is specific to your model. 
#The entry script must understand the data that the model expects and returns.

#The two things you need to accomplish in your entry script are:

#Loading your model (using a function called init())
#Running your model on input data (using a function called run())
#For your initial deployment, use a dummy entry script that prints the data it receives.


import json


def init():
    print("This is init")


def run(data):
    test = json.loads(data)
    print(f"received data {test}")
    return f"test is {test}"

#Save this file as echo_score.py inside of a directory called source_dir. This dummy script returns the data you send to it, so it doesn't use the model. But it is useful for testing that the scoring script is running.

#The following example demonstrates how to create a minimal environment with no pip dependencies,
#using the dummy scoring script you defined above



#The following Python demonstrates how to create a local deployment configuration:





**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
